In [1]:
from matplotlib import pyplot as plt
from PIL import Image
import math
import numpy as np

from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator

from keras.utils import plot_model
from keras import backend


In [2]:
# Keras has a built in ImageDataGenerator class which can load images and manipulate them, but most importantly, keep only the images
# in use in memory. Most of the settings below give the amount of manipulation, 
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range = 0.2,
    height_shift_range=0.2,
    rescale = 1/255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip=True,
    fill_mode = 'nearest'
)

# These iterator objects automatically detect classes, and can be passed to the fit_generator function
fatso_train_it = datagen.flow_from_directory('kitty_images/fatty_train/', class_mode='binary', batch_size=64)
fatso_val_it = datagen.flow_from_directory('kitty_images/fatty_validation/', class_mode='binary', batch_size=64)
fatso_test_it = datagen.flow_from_directory('kitty_images/fatty_test/', class_mode = 'binary', batch_size=64)

# naj_train_it = datagen.flow_from_directory('kitty_images/naj_train/', class_mode='binary', batch_size=64)
# naj_val_it = datagen.flow_from_directory('kitty_images/naj_validation/', class_mode='binary', batch_size=64)
# naj_test_it = datagen.flow_from_directory('kitty_images/naj_test/', class_mode = 'binary', batch_size=64)


Found 4641 images belonging to 2 classes.
Found 530 images belonging to 1 classes.
Found 516 images belonging to 1 classes.


In [ ]:
def make_model():
    conv_model = models.Sequential()
    conv_model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
    conv_model.add(layers.MaxPooling2D((2, 2)))
    conv_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    conv_model.add(layers.MaxPooling2D((2, 2)))
    conv_model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    conv_model.add(layers.MaxPooling2D((2, 2)))
    conv_model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    conv_model.add(layers.MaxPooling2D((2, 2)))
    conv_model.add(layers.Flatten())
    conv_model.add(layers.Dropout(0.5))
    conv_model.add(layers.Dense(512, activation='relu'))
    conv_model.add(layers.Dense(2, activation='softmax'))

    return conv_model

LEARNING_RATE = 1e-4

fatso_model = make_model()
fatso_model.compile(loss='binary_crossentropy',
                   optimizer=optimizers.RMSprop(lr=LEARNING_RATE),
                   metrics=['acc'])

 
# naj_model = make_model()
# naj_model.compile(loss='binary_crossentropy',
#                    optimizer=optimizers.RMSprop(lr=LEARNING_RATE),
#                    metrics=['acc'])


In [ ]:
# Train el models (lets just do fatso first)

fatso_model.fit_generator(fatso_train_it, steps_per_epoch=106, validation_data = fatso_val_it, validation_steps = 40)

loss = fatso_model.evaluate_generator(fatso_test_it,steps=32)

fatso_model.save_weights('fatso_model_weights.h5')